# 3D Solid Mechanics

In [1]:
from netgen.occ import *
from netgen.webgui import Draw as DrawGeo
import ngsolve
ngsolve.__version__

'6.2.2105-64-g8568b5f22'

In [2]:
box = Box((0,0,0), (3,0.6,1))
box.faces.name="outer"
cyl = sum( [Cylinder((0.5+i,0,0.5), Y, 0.25,0.8) for i in range(3)] )
cyl.faces.name="cyl"
geo = box-cyl

find edges between box and cylinder, and build chamfers:

In [3]:
cylboxedges = geo.faces["outer"].edges * geo.faces["cyl"].edges
cylboxedges.name = "cylbox"
geo = geo.MakeChamfer(cylboxedges, 0.03)
geo.faces.Min(X).name = "fix"
geo.faces.Max(X).name = "force"

In [4]:
DrawGeo(geo);

WebGuiWidget(value={'ngsolve_version': 'Netgen x.x', 'mesh_dim': 3, 'mesh_center': [1.5, 0.3, 0.5], 'mesh_radi…

In [5]:
from ngsolve import *
from ngsolve.webgui import Draw
mesh = Mesh(OCCGeometry(geo).GenerateMesh(maxh=0.1)).Curve(3)
Draw (mesh);

WebGuiWidget(value={'ngsolve_version': '6.2.2105-64-g8568b5f22', 'mesh_dim': 3, 'order2d': 2, 'order3d': 2, 'd…

In [6]:
E, nu = 210, 0.2
mu  = E / 2 / (1+nu)
lam = E * nu / ((1+nu)*(1-2*nu))

def Stress(strain):
    return 2*mu*strain + lam*Trace(strain)*Id(3)    

In [7]:
fes = VectorH1(mesh, order=3, dirichlet="fix")
u,v = fes.TnT()
gfu = GridFunction(fes)

with TaskManager():
    a = BilinearForm(InnerProduct(Stress(Sym(Grad(u))), Sym(Grad(v))).Compile()*dx)
    pre = Preconditioner(a, "bddc")
    a.Assemble()

In [8]:
f = LinearForm(1e-3*v[0]*ds("force")).Assemble()

In [9]:
from ngsolve.krylovspace import CGSolver
inv = CGSolver(a.mat, pre, printrates='\r')
gfu.vec.data = inv * f.vec

CG converged in 93 iterations to residual 1.690213829965995e-16


In [10]:
with TaskManager():
    fesstress = MatrixValued(H1(mesh,order=3), symmetric=True)
    gfstress = GridFunction(fesstress)
    gfstress.Interpolate (Stress(Sym(Grad(gfu))))

In [11]:
Draw (5e4*gfu, mesh);

WebGuiWidget(value={'ngsolve_version': '6.2.2105-64-g8568b5f22', 'mesh_dim': 3, 'order2d': 2, 'order3d': 2, 'd…

In [12]:
Draw (Norm(gfstress), mesh, deformation=2*gfu, draw_vol=False, order=3);

WebGuiWidget(value={'ngsolve_version': '6.2.2105-64-g8568b5f22', 'mesh_dim': 3, 'order2d': 3, 'order3d': 2, 'd…